In [1]:
#https://github.com/pytorch/examples/blob/master/mnist/main.py

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [21]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [22]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [34]:
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
seed = 1
log_interval = 10
use_cuda = False
print ("torch.cuda.is_available():", torch.cuda.is_available())

torch.cuda.is_available(): False


In [24]:
torch.manual_seed(seed)

In [35]:
device = torch.device("cuda" if use_cuda else "cpu")
print ("device:", device)

device: cpu


In [26]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [27]:
%timeit
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [37]:
print("type(train_loader):", type(train_loader))

type(train_loader): <class 'torch.utils.data.dataloader.DataLoader'>


In [28]:
%timeit
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)


In [38]:
print("type(test_loader):", type(test_loader))
#https://pytorch.org/docs/master/data.html
#https://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader
#

type(test_loader): <class 'torch.utils.data.dataloader.DataLoader'>


In [29]:
model = Net().to(device)

In [30]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [33]:
%timeit
for epoch in range(1, epochs + 1):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.376790
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.332813
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.291239
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.253216
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.234785
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.259440
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.200771
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.117109
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.170748
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.010879
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.003868
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.714893
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.748160
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.665006
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.777984
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.583380
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.335536
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.286783
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.344140
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.149947
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.226617
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.245688
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.456617
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.661968
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.424896
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.385485
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.337285
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.252275
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.591271
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.320563
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.349734
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.350413
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.481959
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.581771
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.415772
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.264292
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.364824
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.183510
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.227512
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.306094


Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.255632
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.305278
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.188094
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.123647
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.355614
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.277534
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.130193
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.224861
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.446001
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.374752
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.276817
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.239039
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.217002
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.094780
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.215624
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.145369
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.364033
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.202189
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.205632
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.256712


Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.306514
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.121778
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.195158
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.142260
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.485194
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.311035
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.493182
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.092372
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.258737
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.136162
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.240119
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.261939
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.175034
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.091698
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.124109
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.131426
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.223738
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.480439
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.180883
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.266930


Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.125151
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.274086
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.175553
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.383818
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.129618
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.167703
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.190085
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.427089

Test set: Average loss: 0.0610, Accuracy: 9807/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.267243
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.220572
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.152773
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.199300
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.236983
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.143519
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.319411
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.158615
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.234854
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.156039
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.091765
Train Epoch: 

Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.142352
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.066857
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.152450
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.070309
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.141277
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.141554
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.182674
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.160679
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.237048
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.289763
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.213304
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.189766
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.215812
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.167023
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.143792
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.188719
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.129553
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.188428
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.096048
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.263791


In [36]:
print ("type(model):", type(model))
print ("type(device):", type(device))
print ("type(test_loader):", type(test_loader))

type(model): <class '__main__.Net'>
type(device): <class 'torch.Device'>
type(test_loader): <class 'torch.utils.data.dataloader.DataLoader'>
